<a href="https://colab.research.google.com/github/proxai/proxai/blob/main/docs/tutorial_colabs/ProxAI_Advanced_Usage_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🚀 ProxAI Advanced Usage Tutorial 🚀

## 👋 Introduction

Welcome to the ProxAI Advanced Usage Tutorial! This notebook will guide you through some of the advanced features of the ProxAI library. ProxAI unifies AI connections across different providers and offers powerful tools to manage your AI interactions.

In this tutorial, we will cover:
1. ⚡️ Setting up ProxAI in Google Colab
2. 🔋 List Available Models
3. 🤖 Generate Text
4. 🌐 Web Search
5. 📋 Structured Output with Pydantic
6. 🔮 Set Global Model
7. ❤️‍🩹 Check Health
8. 🧪 Experiment Path
9. 🎞️ Logs Management
10. 💾 Cache System - ⭐️ Highly Recommended! ⭐️
11. 🔌 ProxDash Connection
12. 🚦 Feature Mapping Strategy
13. ⚠️ Suppress Provider Errors
14. 📝 Get Current Options
15. 🧩 Using px.Client
16. 🍾 Final Thoughts and Next Steps

Remember, the most effective way to master ProxAI (or any tool!) is to **dive in and use it directly to solve problems.** This tutorial is useful for people who want to formally dive deep into the powerful features of ProxAI.

# 1. ⚡️ Setup in Google Colab
Documentation: [proxai.co/proxai-docs](https://www.proxai.co/proxai-docs)

## 1.1. 💻 Installation

First, let's install the ProxAI library from PyPI.

You can track releases on the [roadmap page](https://www.proxai.co/resources/roadmap) 🗺️.

**Note:** After running the installation cell, you will likely need to **🔄 restart the Colab session** using the button that appears in the output of the cell or by going to `Runtime > Restart session`.

In [ ]:
!pip install proxai

## 1.2. 🔑 API Key Management

ProxAI works with various AI providers. You'll need to add your API keys as secrets in Google Colab. This is the safest way to handle them.

1.  Click on the **🔑 icon (Secrets)** in the left sidebar of Colab.
2.  Add your API keys with the names ProxAI expects (e.g., `OPENAI_API_KEY`, `GEMINI_API_KEY`, `PROXDASH_API_KEY`, etc.). Refer to the [Provider Integrations documentation](https://www.proxai.co/proxai-docs/provider-integrations) for the full list of environment keys.

Run the following cell to load your API keys from Colab secrets into the environment.

<div style="background-color: #ffebee; border-left: 6px solid #f44336; padding: 10px; margin-bottom: 15px;">
  <p style="margin: 0; font-weight: bold; color: #c62828;">🚫 Important Security Note:</p>
  <p style="margin: 0; color: #c62828;">Never directly add API key values as string variables inside the Colab cells. Even after deletion, they can be retrieved from the Colab history.</p>
</div>

In [ ]:
import os
from google.colab import userdata
from dataclasses import asdict
from pprint import pprint

API_KEY_LIST = [
    'ANTHROPIC_API_KEY',
    'CO_API_KEY',
    'DATABRICKS_TOKEN',
    'DATABRICKS_HOST',
    'DEEPSEEK_API_KEY',
    'GEMINI_API_KEY',
    'XAI_API_KEY',
    'HF_TOKEN',
    'MISTRAL_API_KEY',
    'OPENAI_API_KEY',
    'PROXDASH_API_KEY',
]

print("🔐 Attempting to load API keys from Colab secrets...")
for api_key_name in API_KEY_LIST:
  try:
    os.environ[api_key_name] = userdata.get(api_key_name)
    print(f"  ✅ Successfully loaded {api_key_name}")
  except userdata.SecretNotFoundError:
    print(f"  ⚠️ Secret for {api_key_name} not found. Skipping.")
  except Exception as e:
    print(f"  ❌ An error occurred while loading {api_key_name}: {e}")

## 1.3. ▶️ Import ProxAI

Ready to go!

In [ ]:
import proxai as px

# 2. 🔋 List Available Models

Documentation: [proxai.co/proxai-docs/available-models](https://www.proxai.co/proxai-docs/available-models)

## 2.1. 🪛 Simple Usage

Let's list available models in our session! 🎉 \
**Note:** This can take for a while for the first run but the results are cached and it will be fast for other runs.

In [ ]:
provider_models = px.models.list_models()
for provider_model in provider_models:
  print(f'{provider_model.provider:>25} - {provider_model.model}')

## 2.2. 🔭 Different Model Sizes

It is possible to filter out models according to ProxAI sizes.

In [ ]:
provider_models = px.models.list_models(model_size='small')
print('🥚 Small models:')
for provider_model in provider_models:
  print(f'{provider_model.provider:>25} - {provider_model.model}')

provider_models = px.models.list_models(model_size='medium')
print('🐣 Medium models:')
for provider_model in provider_models:
  print(f'{provider_model.provider:>25} - {provider_model.model}')

provider_models = px.models.list_models(model_size='large')
print('🐥 Large models:')
for provider_model in provider_models:
  print(f'{provider_model.provider:>25} - {provider_model.model}')

provider_models = px.models.list_models(model_size='largest')
print('🐓 Largest models of each provider:')
for provider_model in provider_models:
  print(f'{provider_model.provider:>25} - {provider_model.model}')

## 2.3. 📃 Details of Model List

You can get detailed metadata of available models:
* `working_models`: List of working models
* `failed_models`: List of failed models
* `provider_queries`: Dictionary of provider queries that each key has `px.types.ProviderModelType` and each value has `px.types.LoggingRecord`. You can check error messages from these queries.

In [ ]:
model_status = px.models.list_working_models(model_size='small', return_all=True)

print(f'✅ Available models: {len(model_status.working_models)}')
print(f'❌ Failed models: {len(model_status.failed_models)}\n')
errors = []
for provider_model, query in model_status.provider_queries.items():
  if query.response_record.error:
    errors.append((str(provider_model), query.response_record.error))
pprint(errors[:3])

# 3. 🤖 Generate Text

Documentation: [proxai.co/proxai-docs/generate-text](https://www.proxai.co/proxai-docs/generate-text)

## 3.1. 🐶 The Simplest Usage

You can directly call `px.generate_text()` without any additional paramters. ProxAI picks default model or fallback models if default model is not working.

In [ ]:
response = px.generate_text('Hello! Which model are you?')
print(response)

## 3.2. ✏️ Setting Provider Model

There is two different way of directly setting model on `px.generate_text()`
* Tuple with provider and model string
* `px.types.ProviderModelType` value

In [ ]:
print('✏️ Tuple provider_model value:')
response = px.generate_text(
    'Hello! Which model are you?',
    provider_model=('claude', 'haiku-4.5'))
print(response)

print('\n✒️ px.types.ProviderModelType value:')
response = px.generate_text(
    'Hello! Which model are you?',
    provider_model=px.models.get_model('gemini', 'gemini-3-flash'))
print(response)

## 3.3. 👑 System Prompt

You can set system prompt as follows.

In [ ]:
response = px.generate_text(
    'Hello! Which model are you?',
    system="You are an helpful assitant that allways answers in Japan.",
    provider_model=('claude', 'haiku-4.5'))
print(response)

## 3.4. 💬 Message History

It is possible to give the history of conversations to the model via messages. This helps to give context to the model.
* (role=user/assistant, content=text) is a common format for AI provider APIs.
* Some provider uses different tags but you don't need to worry about it. ProxAI handles these integrations.

In [ ]:
response = px.generate_text(
    system="No matter what, always answer with single integer.",
    messages=[
        {"role": "user", "content": "Hello AI Model!"},
        {"role": "assistant", "content": "17"},
        {"role": "user", "content": "How are you today?"},
        {"role": "assistant", "content": "923123"},
        {"role": "user",
         "content": "Can you answer question without any integer?"}
    ],
)
print(response)

## 3.5. ✋ Max Tokens

Limit the token size to avoid cost and delay.

In [ ]:
response = px.generate_text(
    'Can you write all numbers from 1 to 1000?',
    max_tokens=100)
print(response)

## 3.6. 🌡️ Tempreture

If you are looking for more creative answers and more randomness, you can set tempreture to lower values than 1.

In [ ]:
response = px.generate_text(
    'If 5 + 20 would be a poem, what life be look like?',
    temperature=0.01)
print(response)

## 3.7. ☕️ Extensive Return

You can get all details and metadata about query made to the provider by setting `extensive_return` to `True`

In [ ]:
response = px.generate_text(
    'Hello! Which model are you?',
    extensive_return=True)
pprint(asdict(response))

## 3.8. ⚠️ Suppress Provider Errors

If you don't want to raise error when provider fails and just want to continue with error message, you can set `suppress_provider_errors` to `True`.
* Check error message on `logging_record.response_record.error`
* Check error traceback on `logging_record.response_record.error_traceback`

In [ ]:
# Use the mock_failing_provider to demonstrate suppress_provider_errors
response = px.generate_text(
    'If 5 + 20 would be a poem, what life be look like?',
    provider_model=('mock_failing_provider', 'mock_failing_model'),
    suppress_provider_errors=True,
    extensive_return=True)

# No error raised before printing the response or error:
print(f'🤖 Model: {response.query_record.provider_model}')
print(f'💬 Response: {response.response_record.response}')
print(f'❌ Error: {response.response_record.error.strip()}')
print(f'⚠️ Error Traceback:\n{response.response_record.error_traceback.strip()}')

# 4. 🌐 Web Search

Documentation: [proxai.co/proxai-docs/generate-text#web-search](https://www.proxai.co/proxai-docs/generate-text#web-search)

ProxAI supports web search capabilities for models that have this feature. This is useful for questions about recent events, real-time data, or any information that might be beyond the model's training data.

## 4.1. 🔍 Simple Web Search

Enable web search by setting `web_search=True` in your `px.generate_text()` call. This allows the model to search the web for current information.

In [ ]:
# Ask about recent events with web search enabled
response = px.generate_text(
    prompt="What are the latest developments in AI as of this week?",
    web_search=True
)
print(response)

## 4.2. 🔎 Finding Web Search Compatible Models

Not all models support web search. Use the `features` parameter to filter models that support this capability.

In [ ]:
# Find models that support web search
web_search_models = px.models.list_models(features=['web_search'])

print('🌐 Models supporting web search:')
for model in web_search_models:
  print(f'{model.provider:>25} - {model.model}')

# 5. 📋 Structured Output with Pydantic

Documentation: [proxai.co/proxai-docs/generate-text#structured-output](https://www.proxai.co/proxai-docs/generate-text#structured-output)

ProxAI supports structured outputs using Pydantic models. This feature allows you to get type-safe, validated responses from AI models. The model's response will be automatically parsed and validated against your Pydantic schema.

## 5.1. 🏗️ Simple Pydantic Example

Define a Pydantic model and pass it as the `response_format` parameter. The model will return a structured response matching your schema.

In [ ]:
from pydantic import BaseModel

# Define a Pydantic model for the expected response structure
class CityInfo(BaseModel):
    name: str
    country: str
    population: int
    famous_landmark: str

# Get structured output from the AI model
result = px.generate_text(
    prompt="Tell me about Paris, France. Include its population and a famous landmark.",
    response_format=CityInfo
)

# Access the response as a typed object
print(f'🏙️ City: {result.name}')
print(f'🌍 Country: {result.country}')
print(f'👥 Population: {result.population:,}')
print(f'🗼 Famous Landmark: {result.famous_landmark}')

## 5.2. 🔎 Finding Pydantic Compatible Models

Not all models support structured output with Pydantic. Use the `features` parameter to filter models that support this capability.

In [ ]:
# Find models that support Pydantic structured output
pydantic_models = px.models.list_models(features=['response_format::pydantic'])

print('📋 Models supporting Pydantic structured output:')
for model in pydantic_models:
  print(f'{model.provider:>25} - {model.model}')

# 6. 🔮 Set Global Model

Documentation: [proxai.co/proxai-docs/set-global-model](https://www.proxai.co/proxai-docs/set-global-model)

You can set global default model by `px.set_model()` instead of using what ProxAI picks for you. All unspecified `px.generate_text()` calls will use this model.

In [ ]:
# Let's define python method that doesn't specify provider_model
def simple_request():
  return px.generate_text(
      'Hey AI model! This is simple request. Give an answer. Quick!',
  ).strip().replace('\n', ' ')[:80]

# We can change default model by px.set_model
for provider_model in px.models.list_models():
  px.set_model(provider_model)
  response = simple_request()
  print(f'{provider_model} - {response}')

# 7. ❤️‍🩹 Check Health

Documentation: [proxai.co/proxai-docs/check-health](https://www.proxai.co/proxai-docs/check-health)

## 7.1. 🐱 Simple Check

In [ ]:
px.check_health()

## 7.2. 📃 Extensive Return

In [ ]:
model_status = px.check_health(verbose=False, extensive_return=True)
print('--- model_status.working_models:')
pprint(model_status.working_models)
print('--- model_status.failed_models:')
pprint(model_status.failed_models)

# 8. 🧪 Experiment Path

Documentation: [proxai.co/proxai-docs/advanced/experiment-path](https://www.proxai.co/proxai-docs/advanced/experiment-path)

To able unlock experiments path features, please be sure you complated following steps:
1. Open ProxAI account from [proxai.co/signup](https://www.proxai.co/signup)
2. Create ProxAI API key from [proxai.co/dashboard/api-keys](https://www.proxai.co/dashboard/api-keys)
3. Add new API key to 🔑 Colab Secrets from left as `PROXDASH_API_KEY`
4. Run "1.2. 🔑 API Key Management" cell again to load API key

## 8.1. 🐭 Simple Usage with ProxDash

In [ ]:
px.connect(
    experiment_path='colab_experiments/advanced_features/run_1')

px.generate_text('Please recommend me a good movie.')

print('1 - Open ProxDash experiments page: https://proxai.co/dashboard/experiments')
print('2 - Open colab_experiments/advanced_features/run_1 from experiment tree. (Refresh experiments if necessary)')
print('3 - Check logging records tab to see movie recommendation query.')

# 9. 🎞️ Logs Management

Documentation: [proxai.co/proxai-docs/advanced/logs-management](https://www.proxai.co/proxai-docs/advanced/logs-management)

This feature can be more useful in local runs rather than Google Colab.

## 9.1. 🐹 Simple Usage

In [ ]:
import json

px.connect(
    experiment_path='colab_experiments/advanced_features/run_1',
    logging_options=px.LoggingOptions(logging_path='/content/'))

px.generate_text('Hello model!')

print(os.listdir('/content/colab_experiments/advanced_features/run_1'))

with open('/content/colab_experiments/advanced_features/run_1/provider_queries.log', 'r') as f:
  for line in f:
    pprint(json.loads(line))

## 9.2 🕵️ Hide Sensitive Content

You can hide sensitive contents like prompt, response, messages etc. from log files to make proxai more secure.

In [ ]:
px.connect(
    experiment_path='colab_experiments/advanced_features/run_2',
    logging_options=px.types.LoggingOptions(
        logging_path='/content/',
        hide_sensitive_content=True,
    ))

px.generate_text('Hello model!')

print(os.listdir('/content/colab_experiments/advanced_features/run_2'))

print('Following file should\'t show the sensitive information:')
with open('/content/colab_experiments/advanced_features/run_2/provider_queries.log', 'r') as f:
  for line in f:
    pprint(json.loads(line))

## 9.3. 🖥️ Stdout

There is option for printing all logs to the stdout. It is useful for debugging cases.

In [ ]:
px.connect(
    experiment_path='colab_experiments/advanced_features/run_2',
    logging_options=px.types.LoggingOptions(
        logging_path='/content/',
        stdout=True,
    ))

print('You should be able to see logging record on cell output for following:')
response = px.generate_text('Hello model!')

# 10. 💾 Cache System - ⭐️ Highly Recommended! ⭐️

Documentation: [proxai.co/proxai-docs/advanced/cache-system](https://www.proxai.co/proxai-docs/advanced/cache-system)

This feature is very useful at development stage. Without this feature, experiments can get very painful very easily.

First, let's define simple method to get response and duration as in following section.

In [ ]:
import random
import time

def test_cache():
  fixed_int = random.randint(10000, 20000)

  def test_prompt():
    start = time.time()
    response = px.generate_text(
        'Can you pick 100 different random positive integers which are less '
        f'than {fixed_int}? Can you also explain why you picked these numbers? '
        'Please think deeply about your decision and answer accordingly. '
        'Start your sentence with random simple poem.',
        temperature=0.3)
    duration = time.time() - start
    response = response.strip().replace('\n', ' ')[:80]
    return response, duration

  for i in range(1, 7):
    response, duration = test_prompt()
    print(f'{i}: {duration:.3f} sec - {response}')

# Also, set use simpler model:
px.set_model(('gemini', 'gemini-3-flash'))

## 10.1. 🐰 Simple Usage

Following example shows how to use simple query cache by only setting cache_path.
* All responses returned from cache after first query.
* First query takes longer than other queries.

In [ ]:
px.connect(
    experiment_path='colab_experiments/advanced_features/run_3',
    cache_options=px.CacheOptions(cache_path='/content/'))

test_cache()

## 10.2. 🕶️ Unique Response Limit

If you want more diverse results, you can set `unique_response_limit` option.
* This ensures that it makes at least `unique_response_limit` actual provider queries.
* Cache responses returned in round robin fashion.

Following examples shows that first three responses are from provider and takes longer than other three responses.

In [ ]:
px.connect(
    experiment_path='colab_experiments/advanced_features/run_3',
    cache_options=px.CacheOptions(
        cache_path='/content/',
        unique_response_limit=3
    ))

test_cache()

## 10.3 🏓 Skip Cache

It is possible to skip cache and ensure actual provider queries are made. Set `use_cache=False` for `px.generate_text()` method. This ensures for that generate text query, result is always from provider.

In [ ]:
px.connect(
    experiment_path='colab_experiments/advanced_features/run_3',
    cache_options=px.CacheOptions(cache_path='/content/'))

response = px.generate_text(
    'Hello model!',
    extensive_return=True)
print(response.response_source)

response = px.generate_text(
    'Hello model!',
    extensive_return=True)
print(response.response_source)

response = px.generate_text(
    'Hello model!',
    use_cache=False,
    extensive_return=True)
print(response.response_source)

## 10.4. 🛀 Clear Cache and Override Params

It is possible clear cache on connect to ensure session doesn't have any cache.\
Let's also override the `unique_response_limit` on `px.generate_text` observe more control.

In [ ]:
px.connect(
    experiment_path='colab_experiments/advanced_features/run_3',
    cache_options=px.CacheOptions(
        cache_path='/content/',
        unique_response_limit=3,
        clear_query_cache_on_connect=True,
    ))

response = px.generate_text(
    'Hello model!',
    unique_response_limit=1,
    extensive_return=True)
print(response.response_source)

response = px.generate_text(
    'Hello model!',
    unique_response_limit=1,
    extensive_return=True)
print(response.response_source)

response = px.generate_text(
    'Hello model!',
    unique_response_limit=1,
    extensive_return=True)
print(response.response_source)

# 11. 🔌 ProxDash Connection

Documentation: [proxai.co/proxai-docs/advanced/proxdash-connection](https://www.proxai.co/proxai-docs/advanced/proxdash-connection)

There are number of advantages to use ProxAI with ProxDash. Please, refer advantages on [proxai.co](https://www.proxai.co/) and [resources](https://www.proxai.co/resources/why)

In "8. 🧪 Experiment Path" section, we already used proxdash. Please, check the steps required over there if you skipped that section.

## 11.1. 🦊 Simple Usage

In [ ]:
px.connect()

print('By default, this should appear on ProxDash logging history if ProxDash\n'
      'API key set in colab.\n\n'
      'Please check https://www.proxai.co/dashboard/logging if you can see\n'
      'following query on ProxDash.')
response = px.generate_text('This is temp proxdash test')

## 11.2. 📂 Setting Experiment Path

In [ ]:
px.connect(
    experiment_path='colab_experiments/advanced_features/run_4')

print('By default, this should appear on ProxDash logging history and\n'
      'experiments with provided experiment path.\n\n'
      'Please check https://www.proxai.co/dashboard/experiments if you\n'
      'can see following experiment path:\n'
      '> colab_experiments/advanced_features/run_4\n\n'
      'If you open this experiment and go logging record tab, you should be\n'
      'able to see following query on ProxDash.')
response = px.generate_text('This is temp proxdash experiment path test')

## 11.3. 🕵️ Hide Sensitive Content

Normally, ProxDash respects the privacy level you set on the API key generation page. However, you still have control over the fields you want to send to ProxDash in case:
* API key has the permission but you don't want to send some fields to ProxDash anyway.
* API key doesn't have the permission and you want to ensure to block the fields rather relying on the ProxDash permission level.

To do this, you can use the hide_sensitive_content option in the proxdash_options parameter.

In [ ]:
px.connect(
    experiment_path='colab_experiments/advanced_features/run_4',
    proxdash_options=px.ProxDashOptions(
        hide_sensitive_content=True
    ))

print('Following query record should appear on ProxDash logging history but\n'
      'the content of the prompt and response cannot be visible.\n'
      'Please check the latest logging record on '
      'https://www.proxai.co/dashboard/logging to confirm that.')
response = px.generate_text(
    'This record should appear on ProxDash but the prompt content '
    'and the response content from AI provider shouldn\'t appear on ProxDash.')

## 11.4. 📣 Print ProxDash Connection Logs

In [ ]:
print('You should be able to see ProxDash connection status:')
px.reset_state()
px.connect(
    experiment_path='colab_experiments/advanced_features/run_4',
    proxdash_options=px.ProxDashOptions(
        stdout=True
    ))

## 11.5. 🚧 Disable ProxDash

You can remove `PROXDASH_API_KEY` from environment variables to disable ProxDash but there is simpler way:

In [ ]:
px.connect(
    proxdash_options=px.ProxDashOptions(
        stdout=True,
        disable_proxdash=True,
    ))

print('Following record should not appear on ProxDash logging history:\n'
      'https://www.proxai.co/dashboard/logging')
reponse = px.generate_text('This prompt should not appear on proxdash')

# 12. 🚦 Feature Mapping Strategy

Documentation: [proxai.co/proxai-docs/advanced/feature-mapping-strategy](https://www.proxai.co/proxai-docs/advanced/feature-mapping-strategy)

Not all models support all features. ProxAI provides two strategies for handling feature compatibility:

* **BEST_EFFORT** (default): Attempts to map features even if not fully supported. For example, simulating system messages for models that don't natively support them.
* **STRICT**: Requires exact feature support. Raises errors if the requested feature is not supported by the model.

Set `feature_mapping_strategy=px.types.FeatureMappingStrategy.STRICT` on `px.connect()` to enforce strict feature requirements.

In [ ]:
px.connect(
    experiment_path='colab_experiments/advanced_features/run_6',
    feature_mapping_strategy=px.types.FeatureMappingStrategy.STRICT)

try:
  px.generate_text(
      'Create me a simple poem about birds.',
      provider_model=('openai', 'o1'),
      temperature=0.3)
except Exception as e:
  print(
      'This query raises error because temperature feature is not supported on '
      'OpenAI\'s o1 model.')
  print(f'Error: {e}')

# 13. ⚠️ Suppress Provider Errors

Documentation: [https://www.proxai.co/proxai-docs/advanced/suppress-provider-errors](https://www.proxai.co/proxai-docs/advanced/suppress-provider-errors)

Instead of raising AI provider errors, you can get `logging_record.response_query.error` and `logging_record.response_query.error_traceback` by setting `suppress_provider_errors=True`.

This allows you to continue the processing and check what errors are happening from ProxDash.

In [ ]:
px.connect(suppress_provider_errors=True)

# Use the mock_failing_provider to demonstrate suppress_provider_errors
response = px.generate_text(
    'If 5 + 20 would be a poem, what life be look like?',
    provider_model=('mock_failing_provider', 'mock_failing_model'),
    extensive_return=True)

# No error raised before printing the response or error:
print(f'🤖 Model: {response.query_record.provider_model}')
print(f'💬 Response: {response.response_record.response}')
print(f'❌ Error: {response.response_record.error.strip()}')
print(f'⚠️ Error Traceback:\n{response.response_record.error_traceback.strip()}')

# 14. 📝 Get Current Options

Documentation: [proxai.co/proxai-docs/advanced/get-current-options](https://www.proxai.co/proxai-docs/advanced/get-current-options)

## 14.1. 🐻 Simple Usage

In [ ]:
pprint(asdict(px.get_current_options()))

## 14.2. 🦁 Also Another Simple Example

In [ ]:
px.connect(
    experiment_path='colab_experiments/advanced_features/run_5',
    logging_options=px.LoggingOptions(
        logging_path='/content/',
        stdout=True,
        hide_sensitive_content=True),
    cache_options=px.CacheOptions(
        cache_path='/content/',
        retry_if_error_cached=True,
        unique_response_limit=3),
    proxdash_options=px.ProxDashOptions(
        stdout=True,
        hide_sensitive_content=True),
    feature_mapping_strategy=px.types.FeatureMappingStrategy.STRICT,
    allow_multiprocessing=False,
    suppress_provider_errors=True)

pprint(px.get_current_options(json=True))

# 15. 🧩 Using px.Client

Documentation: [proxai.co/proxai-docs/advanced/client](https://www.proxai.co/proxai-docs/advanced/client)

While the global `px.connect()` and `px.generate_text()` functions work great for most use cases, sometimes you need more control. The `px.Client` class lets you create independent client instances with their own configurations.

This is useful when you want to:
* Use different models or settings for different parts of your application
* Run multiple experiments simultaneously with different configurations
* Have isolated state between different components

## 15.1. 🐧 Creating a Client Instance

Create an independent client with its own configuration. The client supports all the same options as `px.connect()`.

In [ ]:
# Create an independent client instance with its own configuration
client = px.Client(
    experiment_path='colab_experiments/client_demo/experiment_1',
    cache_options=px.CacheOptions(cache_path='/content/'),
    logging_options=px.LoggingOptions(stdout=True),
)

# Set a model for this client
client.set_model(('gemini', 'gemini-3-flash'))

# Generate text using this client
response = client.generate_text(prompt='Hello from px.Client!')
print(f'🧩 Client response: {response}')

## 15.2. 🐼 Using Multiple Clients

You can use both the global `px` functions and multiple `px.Client` instances simultaneously. Each operates independently with its own configuration.

In [ ]:
# Configure the global client
px.connect(experiment_path='colab_experiments/global_experiment')
px.set_model(('gemini', 'gemini-3-flash'))

# Create a separate client with different configuration
claude_client = px.Client(
    experiment_path='colab_experiments/claude_experiment',
)
claude_client.set_model(('claude', 'haiku-4.5'))

# Use both clients independently
print('🌍 Global px client:')
global_response = px.generate_text('What is 2 + 2?', extensive_return=True)
print(f'  Model: {global_response.query_record.provider_model}')
print(f'  Response: {global_response.response_record.response.strip()}')

print('\n🧩 Claude client:')
claude_response = claude_client.generate_text('What is 2 + 2?', extensive_return=True)
print(f'  Model: {claude_response.query_record.provider_model}')
print(f'  Response: {claude_response.response_record.response.strip()}')

# 🍾 Final Thoughts and Next Steps

### 🎉 Congratulations🎉

Congratulations on completing the ProxAI Advanced Usage Tutorial!

### 🚀 Create, Innovate, and Share!

The real magic happens when you start building! We wholeheartedly encourage you to:
* 🥇 **Develop cool scripts, intricate code examples, and innovative projects** using ProxAI.
* 🏆 **Share your creations!** Whether it's with the ProxAI community, on your blog, in forums, or with colleagues, your examples can inspire and help others.

### 🌱 Learn by Doing

We've covered a lot of ground, and going through each feature meticulously one by one can sometimes feel daunting. Remember, the most effective way to master ProxAI (or any tool!) is to **dive in and use it directly to solve problems.**

### 🤝 Contribute to ProxAI

ProxAI is an open-source project, and its strength grows with its community. We warmly invite you to contribute!
* **Report bugs or suggest new features:** Your feedback is invaluable. (See [Reporting Bugs & Feature Requests](https://www.proxai.co/resources/community))
* **Improve documentation:** Help us make the docs clearer and more comprehensive.
* **Write code:** Contribute fixes, new features, or new provider integrations.

Check out our **[ProxAI GitHub repository](https://github.com/proxai/proxai)** and the [Contribution Guidelines](https://www.proxai.co/resources/community/guidelines) to get started.

### 📞 Get in Touch

We're here to help and love hearing from our users!

* **Discord Community:** For real-time chat, support, and discussions: [discord.gg/QhrDkzMHrP](https://discord.gg/QhrDkzMHrP)
* **GitHub Issues:** For technical questions, bug reports, and feature requests: [github.com/proxai/proxai/issues](https://github.com/proxai/proxai/issues)
* **Email Contacts:**
    * Feedback & Feature Suggestions: [feedback@proxai.co](feedback@proxai.co) ⭐️
    * Development & Contribution Support: [dev@proxai.co](dev@proxai.co)
    * General Community Inquiries: [community@proxai.co](community@proxai.co)

Thank you for learning with ProxAI. We can't wait to see what you build! 🚀